In [13]:
import gzip, numpy, torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score

In [2]:
batch_size = 5
nb_epochs = 100
eta = 0.01

In [3]:
# Chargement des données
((data_train, label_train), (data_test, label_test)) = torch.load(gzip.open('mnist.pkl.gz'))

C:\Users\DELL\AppData\Local\Temp\ipykernel_32808\1883767992.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  ((data_train, label_train), (data_test, label_test)) = torch.

In [4]:
data_train, data_validation, label_train, label_validation = train_test_split(data_train, label_train, test_size=0.2)

In [5]:
# on crée les lecteurs de données
train_dataset = torch.utils.data.TensorDataset(data_train,label_train)
test_dataset = torch.utils.data.TensorDataset(data_test,label_test)
valid_dataset = torch.utils.data.TensorDataset(data_validation,label_validation)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=1, shuffle=False)
valid_loader = torch.utils.data.DataLoader(valid_dataset, batch_size=1, shuffle=False)

In [6]:
class DeepNetwork(nn.Module):
    
    def __init__(self,input_size, hidden1_size, hidden2_size, output_size):
        super(DeepNetwork,self).__init__()
        self.hidden1 = nn.Linear(input_size,hidden1_size)
        self.relu = nn.ReLU()
        self.dropout1 = nn.Dropout(0.5)
        self.hidden2 = nn.Linear(hidden1_size, hidden2_size)
        self.relu = nn.ReLU()
        self.dropout2 = nn.Dropout(0.5)
        self.output = nn.Linear(hidden2_size,output_size)
        
    def forward(self,x):
        out = self.hidden1(x)
        out = self.relu(out)
        out = self.dropout1(out)
        out = self.hidden2(out)
        out = self.relu(out)
        out = self.dropout2(out)
        out = self.output(out)
        return out

In [7]:
input_size = 784
hidden1_size = 150
hidden2_size = 150
output_size = 10

In [8]:
model = DeepNetwork(input_size, hidden1_size, hidden2_size, output_size)

In [9]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=eta)

In [10]:
best_val_loss = float('inf')
p = 5  
t = 0

In [11]:
for epoch in range(nb_epochs):
    model.train()
    running_loss = 0
    for images ,labels in train_loader:
        optimizer.zero_grad()
        outputs = model(images)
        loss=criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
    print(f"Époch {epoch+1}/{nb_epochs}, Loss = {loss.item():.4f}")
    
    model.eval()
    val_loss = 0
    with torch.no_grad():
        for images, labels in valid_loader:
            outputs = model(images)
            loss = criterion(outputs, labels)
            val_loss += loss.item()
    
    val_loss /= len(valid_loader)
    print(f"Epoch {epoch+1}, Validation Loss: {val_loss:.4f}")


    if val_loss < best_val_loss:
        best_val_loss = val_loss
        t = 0
    else:
        t += 1
    
    if t >= p:
        print("Early stopping!")
        break
    

Époch 1/100, Loss = 0.3824
Epoch 1, Validation Loss: 0.2509
Époch 2/100, Loss = 0.0120
Epoch 2, Validation Loss: 0.1764
Époch 3/100, Loss = 0.0594
Epoch 3, Validation Loss: 0.1484
Époch 4/100, Loss = 0.0191
Epoch 4, Validation Loss: 0.1302
Époch 5/100, Loss = 0.1376
Epoch 5, Validation Loss: 0.1165
Époch 6/100, Loss = 0.1795
Epoch 6, Validation Loss: 0.1125
Époch 7/100, Loss = 0.0256
Epoch 7, Validation Loss: 0.1070
Époch 8/100, Loss = 0.1492
Epoch 8, Validation Loss: 0.1019
Époch 9/100, Loss = 0.0124
Epoch 9, Validation Loss: 0.0953
Époch 10/100, Loss = 0.0092
Epoch 10, Validation Loss: 0.0954
Époch 11/100, Loss = 0.1075
Epoch 11, Validation Loss: 0.0938
Époch 12/100, Loss = 0.0018
Epoch 12, Validation Loss: 0.0947
Époch 13/100, Loss = 0.7098
Epoch 13, Validation Loss: 0.0901
Époch 14/100, Loss = 0.0143
Epoch 14, Validation Loss: 0.0883
Époch 15/100, Loss = 0.0068
Epoch 15, Validation Loss: 0.0889
Époch 16/100, Loss = 0.1296
Epoch 16, Validation Loss: 0.0880
Époch 17/100, Loss = 0.000

In [17]:
model.eval()
acc=0
all_labels = []
all_preds = []
with torch.no_grad():
    for images, labels in test_loader:
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        label_multiclas = torch.argmax(labels, dim=1)
        all_labels.extend(label_multiclas.cpu().numpy())
        all_preds.extend(predicted.cpu().numpy())

In [18]:
accuracy = accuracy_score(all_labels, all_preds)
precision = precision_score(all_labels, all_preds, average='macro')
recall = recall_score(all_labels, all_preds, average='macro')
f1 = f1_score(all_labels, all_preds, average='macro')

In [19]:
print(f"Test Accuracy: {accuracy * 100:.2f}%")
print(f"Precision: {precision:.2f}")
print(f"Recall: {recall:.2f}")
print(f"F1-Score: {f1:.2f}")

Test Accuracy: 97.81%
Precision: 0.98
Recall: 0.98
F1-Score: 0.98


In [45]:
'''Deep Network : 

batch_size = 5
nb_epochs = 100
eta = 0.00001

input_size = 784
hidden1_size = 128
hidden2_size = 128
output_size = 10

Test Accuracy: 76.40285583496094%

iterartion 2 : 

batch_size = 5
nb_epochs = 100
eta = 0.00001

input_size = 784
hidden1_size = 60
hidden2_size = 60
output_size = 10

Test Accuracy: 77.44285583496094%

iterartion 3 : 

batch_size = 5
nb_epochs = 100
eta = 0.00001

input_size = 784
hidden1_size = 30
hidden2_size = 30
output_size = 10

Test Accuracy: 66.44285583496094%

iterartion 4 : 

batch_size = 5
nb_epochs = 100
eta = 0.01

input_size = 784
hidden1_size = 150
hidden2_size = 150
output_size = 10

Test Accuracy: Test Accuracy: 97.9142837524414%'''

'Deep Network : \n\nbatch_size = 5\nnb_epochs = 100\neta = 0.00001\n\ninput_size = 784\nhidden1_size = 128\nhidden2_size = 128\noutput_size = 10\n\nTest Accuracy: 76.40285583496094%\n\niterartion 2 : \n\nbatch_size = 5\nnb_epochs = 100\neta = 0.00001\n\ninput_size = 784\nhidden1_size = 60\nhidden2_size = 60\noutput_size = 10\n\nTest Accuracy: 77.44285583496094%\n\niterartion 3 : \n\nbatch_size = 5\nnb_epochs = 100\neta = 0.00001\n\ninput_size = 784\nhidden1_size = 30\nhidden2_size = 30\noutput_size = 10\n\nTest Accuracy: 66.44285583496094%\n\niterartion 4 : \n\nbatch_size = 5\nnb_epochs = 100\neta = 0.01\n\ninput_size = 784\nhidden1_size = 150\nhidden2_size = 150\noutput_size = 10\n\nTest Accuracy: Test Accuracy: 97.9142837524414%'